In [1]:
from pyspark.sql import SparkSession

"""make an instance of a SparkSession called 'spark'."""
spark = SparkSession.builder.master('local').getOrCreate()


22/07/01 12:41:47 WARN Utils: Your hostname, fossa-dsa-001 resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlp3s0)
22/07/01 12:41:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/01 12:41:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [33]:
# set our path
data_file = 'data/coffee.csv'
# set schema
customSchema = 'date date, open float, high float, low float, close float, volume float, currency string'
# read our csv into a spark df
sdf = spark.read.csv(data_file, schema = customSchema, header = True)
sdf.describe

<bound method DataFrame.describe of DataFrame[date: date, open: float, high: float, low: float, close: float, volume: float, currency: string]>

In [39]:
# Add a column to the DataFrame where the values are the difference between 'Open' and 'Close'.
from pyspark.sql.functions import *
sdf = sdf.withColumn('open2close',
                (sdf.open - sdf.close))
# Add a column to the DataFrame where the values are the difference between 'High' and 'Low'.
sdf = sdf.withColumn('high2low',
                (sdf.high - sdf.close))
# Add a column to the DataFrame where the values are 'True' if the volume for that day was 100 or above, and otherwise 'False'.
sdf = sdf.withColumn('did_well',
                     when((sdf.volume > 99), lit(True))
                     .otherwise(lit(False)))
# add another column that contains the absolute values of the numbers in that column.
sdf = sdf.withColumn('day_abs', abs(sdf.open2close))
sdf.show()

+----------+------+------+------+------+-------+--------+-----------+----------+--------+----------+
|      date|  open|  high|   low| close| volume|currency| open2close|  high2low|did_well|   day_abs|
+----------+------+------+------+------+-------+--------+-----------+----------+--------+----------+
|2000-01-03|122.25| 124.0| 116.1| 116.5| 6640.0|     USD|       5.75|       7.5|    true|      5.75|
|2000-01-04|116.25| 120.5|115.75|116.25| 5492.0|     USD|        0.0|      4.25|    true|       0.0|
|2000-01-05| 115.0| 121.0| 115.0| 118.6| 6165.0|     USD| -3.5999985| 2.4000015|    true| 3.5999985|
|2000-01-06| 119.0| 121.4| 116.5|116.85| 5094.0|     USD|  2.1500015|  4.550003|    true| 2.1500015|
|2000-01-07|117.25|117.75| 113.8|114.15| 6855.0|     USD|  3.0999985| 3.5999985|    true| 3.0999985|
|2000-01-10| 123.5| 126.0| 116.7|117.55| 7499.0|     USD|   5.949997|  8.449997|    true|  5.949997|
|2000-01-11| 115.5|118.25| 115.5| 117.8| 3976.0|     USD|  -2.300003|0.44999695|    true|  

In [51]:
net_sales = mean(sdf.open + sdf.high + sdf.low + sdf.close) * sdf.volume
# print(net_sales)
